In [2]:
%load_ext autoreload
%autoreload 2

In [17]:
import numpy as np
import scipy
import sys
import pdb
import matplotlib.pyplot as plt
from glob import glob
from pyuoi.linear_model.var import VAR
from tqdm import tqdm
import pandas as pd
from neurosim.models.var import VAR as VARss
from copy import deepcopy

In [15]:
sys.path.append('../..')
from loaders import load_sabes
from subspaces import estimate_autocorrelation
from utils import apply_df_filters

### Fitting (Sabes)

In [53]:
data_files = glob('/mnt/Secondary/data/sabes/*.mat')

In [55]:
# Check for non-empty S1 indices
valid_data_files = [data_file for data_file in data_files if load_sabes(data_file, region='S1')['spike_rates'].shape[-1] != 0 ]

Processing spikes


100%|██████████| 1/1 [00:00<00:00, 10591.68it/s]


Processing spikes


100%|██████████| 1/1 [00:16<00:00, 16.54s/it]


Processing spikes


...

100%|██████████| 1/1 [00:16<00:00, 16.10s/it]


Processing spikes


100%|██████████| 1/1 [00:10<00:00, 10.74s/it]


Processing spikes


100%|██████████| 1/1 [00:21<00:00, 21.09s/it]


In [ ]:
M1_size = []
S1_size = []
loco_files = glob.glob('/mnt/Secondary/data')

In [ ]:
# Task: Setup a script that can, using MPI, scalably do the VAR fits on cross validated 

In [57]:
valid_data_files

['/mnt/Secondary/data/sabes/indy_20160426_01.mat',
 '/mnt/Secondary/data/sabes/loco_20170210_03.mat',
 '/mnt/Secondary/data/sabes/loco_20170213_02.mat',
 '/mnt/Secondary/data/sabes/loco_20170214_02.mat',
 '/mnt/Secondary/data/sabes/loco_20170215_02.mat',
 '/mnt/Secondary/data/sabes/loco_20170216_02.mat',
 '/mnt/Secondary/data/sabes/loco_20170217_02.mat',
 '/mnt/Secondary/data/sabes/loco_20170227_04.mat',
 '/mnt/Secondary/data/sabes/loco_20170228_02.mat',
 '/mnt/Secondary/data/sabes/loco_20170301_05.mat',
 '/mnt/Secondary/data/sabes/loco_20170302_02.mat']

In [6]:
# For each data file, fit OLS VAR(1), VAR(3), VAR(5) on *both* M1 and S1, as well as M1 and S1 jointly. 

In [7]:
# Also need to compare VAR performance to SSID in neural data

In [8]:
from sklearn.model_selection import KFold

In [9]:
results_list = []
for data_file in tqdm(data_files[0:5]):
    
    dat_M1 = load_sabes(data_file)
    dat_S1 = load_sabes(data_file, region='S1')

    yM1 = np.squeeze(dat_M1['spike_rates'])
    yS1 = np.squeeze(dat_S1['spike_rates'])
    yjoint = np.hstack([yM1, yS1])

    fold_idx = 0 

    # Cross-validated marginal models
    for train_idxs, test_idxs in KFold(5).split(yM1):

        ccm_trainM1 = estimate_autocorrelation(yM1[train_idxs], 5)
        ccm_testM1 = estimate_autocorrelation(yM1[test_idxs], 5)
        ccm_trainS1 = estimate_autocorrelation(yS1[train_idxs], 5)
        ccm_testS1 = estimate_autocorrelation(yS1[test_idxs], 5)

        ccm_train_joint = estimate_autocorrelation(yjoint[train_idxs], 5)
        ccm_test_joint = estimate_autocorrelation(yjoint[test_idxs], 5)

        for var_order in [1, 2, 3, 4, 5]:
            
            results = {}
            results['data_file'] = data_file
            results['fold_idx'] = fold_idx
            results['var_order'] = var_order
            results['ccm_trainM1'] = ccm_trainM1
            results['ccm_testM1'] = ccm_testM1
            results['ccm_trainS1'] = ccm_trainS1 
            results['ccm_testS1'] = ccm_testS1
            results['ccm_train_joint'] = ccm_train_joint
            results['ccm_test_joint'] = ccm_test_joint

            varmodel = VAR(estimator='ols', order=var_order)
            varmodel.fit(yM1[train_idxs])
            results['m1_coef'] = varmodel.coef_

            varmodel.fit(yS1[train_idxs])
            results['s1_coef'] = varmodel.coef_

            varmodel.fit(yjoint[train_idxs])
            results['joint_coef'] = varmodel.coef_
            results_list.append(results)

        fold_idx += 1   

  0%|          | 0/5 [00:00<?, ?it/s]

Processing spikes


100%|██████████| 1/1 [00:16<00:00, 16.23s/it]


Processing spikes


100%|██████████| 1/1 [00:15<00:00, 15.98s/it]
../../cov_estimation.py:142: RuntimeWarning: invalid value encountered in double_scalars
  ccm_sample_normalized[k, i, j] /= np.sqrt(ccm_sample[0, i, i] * ccm_sample[0, j, j])
Rows of VAR matrix processed: 186it [00:11, 16.51it/s]
Rows of VAR matrix processed: 86it [00:01, 46.36it/s]
Rows of VAR matrix processed: 272it [00:28,  9.44it/s]
Rows of VAR matrix processed: 186it [00:25,  7.35it/s]
Rows of VAR matrix processed: 86it [00:03, 22.09it/s]
Rows of VAR matrix processed: 272it [01:01,  4.45it/s]
Rows of VAR matrix processed: 186it [00:42,  4.38it/s]
Rows of VAR matrix processed: 86it [00:07, 11.52it/s]
Rows of VAR matrix processed: 272it [01:58,  2.29it/s]
Rows of VAR matrix processed: 186it [01:06,  2.79it/s]
Rows of VAR matrix processed: 86it [00:10,  8.13it/s]
Rows of VAR matrix processed: 272it [03:02,  1.49it/s]
Rows of VAR matrix processed: 186it [01:36,  1.93it/s]
Rows of VAR matrix processed: 86it [00:13,  6.50it/s]
Rows of VAR m

...

100%|██████████| 1/1 [00:00<00:00, 34663.67it/s]
../../cov_estimation.py:142: RuntimeWarning: invalid value encountered in double_scalars
  ccm_sample_normalized[k, i, j] /= np.sqrt(ccm_sample[0, i, i] * ccm_sample[0, j, j])
Rows of VAR matrix processed: 173it [00:17,  9.76it/s]
Rows of VAR matrix processed: 173it [00:17,  9.88it/s]
Rows of VAR matrix processed: 173it [00:40,  4.26it/s]
Rows of VAR matrix processed: 173it [00:40,  4.24it/s]
Rows of VAR matrix processed: 173it [01:02,  2.76it/s]
Rows of VAR matrix processed: 173it [01:02,  2.78it/s]
Rows of VAR matrix processed: 173it [01:44,  1.66it/s]
Rows of VAR matrix processed: 173it [01:45,  1.65it/s]
Rows of VAR matrix processed: 173it [02:22,  1.22it/s]
Rows of VAR matrix processed: 173it [02:22,  1.21it/s]
Rows of VAR matrix processed: 173it [00:18,  9.27it/s]
Rows of VAR matrix processed: 173it [00:18,  9.50it/s]
Rows of VAR matrix processed: 173it [00:40,  4.28it/s]
Rows of VAR matrix processed: 173it [00:41,  4.22it/s]
Rows 

Processing spikes


100%|██████████| 1/1 [00:13<00:00, 13.50s/it]


Processing spikes


100%|██████████| 1/1 [00:00<00:00, 34663.67it/s]
Rows of VAR matrix processed: 97it [00:01, 52.02it/s]
Rows of VAR matrix processed: 97it [00:01, 51.60it/s]
Rows of VAR matrix processed: 97it [00:04, 22.47it/s]
Rows of VAR matrix processed: 97it [00:04, 20.91it/s]
Rows of VAR matrix processed: 97it [00:07, 12.56it/s]
Rows of VAR matrix processed: 97it [00:07, 12.47it/s]
Rows of VAR matrix processed: 97it [00:10,  8.89it/s]
Rows of VAR matrix processed: 97it [00:10,  8.98it/s]
Rows of VAR matrix processed: 97it [00:14,  6.92it/s]
Rows of VAR matrix processed: 97it [00:14,  6.64it/s]
Rows of VAR matrix processed: 97it [00:02, 46.37it/s]
Rows of VAR matrix processed: 97it [00:02, 46.27it/s]
Rows of VAR matrix processed: 97it [00:04, 20.78it/s]
Rows of VAR matrix processed: 97it [00:04, 20.78it/s]
Rows of VAR matrix processed: 97it [00:08, 12.12it/s]
Rows of VAR matrix processed: 97it [00:07, 12.36it/s]
Rows of VAR matrix processed: 97it [00:11,  8.69it/s]
Rows of VAR matrix processed: 97i

In [14]:
len(results_list)

125

In [10]:
import pickle

In [11]:
with open('sabes_var_fits1.dat', 'wb') as f:
    f.write(pickle.dumps(results_list))

In [13]:
df = pd.DataFrame(results_list)

In [36]:
# Model selection stpep. We select based on cross-validated CCM estimation error. First set of questions: Does the model based off the joint regression do better at CCM error than the marginal models?
results_list = []

etrain_M1 = np.zeros((len(data_files[0:5]), 5, 5, 5))
etest_M1 = np.zeros((len(data_files[0:5]), 5, 5, 5))
etrain_S1 = np.zeros((len(data_files[0:5]), 5, 5, 5))
etest_S1 = np.zeros((len(data_files[0:5]), 5, 5, 5))

etrain_joint = np.zeros((len(data_files[0:5]), 5, 5, 5))
etest_joint = np.zeros((len(data_files[0:5]), 5, 5, 5))

etrain_jointM1 = np.zeros((len(data_files[0:5]), 5, 5, 5))
etest_jointM1 = np.zeros((len(data_files[0:5]), 5, 5, 5))
etrain_jointS1 = np.zeros((len(data_files[0:5]), 5, 5, 5))
etest_jointS1 = np.zeros((len(data_files[0:5]), 5, 5, 5))

for i, data_file in enumerate(data_files[0:1]):
    for j, fold_idx in enumerate(range(5)):
        for k, var_order in enumerate([1, 2, 3, 4, 5]):
            df_ = apply_df_filters(df, data_file=data_file, fold_idx=fold_idx, var_order=var_order)
            assert(df_.shape[0]) == 1
            df_ = df_.iloc[0]

            # Marginal and joint autocorrelations 
            varss_M1 = VARss(df_['m1_coef'])
            varss_S1 = VARss(df_['s1_coef'])
            varss_joint = VARss(df_['joint_coef'])


            if i == 1:
                pdb.set_trace()

            M1_proj = np.block([[np.eye(varss_M1.C.shape[0]), np.zeros((varss_M1.C.shape[0], varss_S1.C.shape[0]))]])
            S1_proj = np.block([[np.zeros((varss_S1.C.shape[0], varss_M1.C.shape[0])), np.eye(varss_S1.C.shape[0])]])





            etrain_M1[i, j, k] = [np.linalg.norm(df_['ccm_trainM1'][idx] - varss_M1.autocorrelation(5)[idx]) for idx in range(5)]
            etest_M1[i, j, k] = [np.linalg.norm(df_['ccm_testM1'][idx] - varss_M1.autocorrelation(5)[idx]) for idx in range(5)]
# Next question: We need to parametrically vary over the time lags that we use to regress the past of one area onto another
            if df_['ccm_trainS1'].shape[-1] == 0: 
                etrain_S1[i, j, k] = 5*[np.nan]
                etest_S1[i, j, k] = 5*[np.nan]
            else:           
                etrain_S1[i, j, k] = [np.linalg.norm(df_['ccm_trainS1'][idx] - varss_S1.autocorrelation(5)[idx]) for idx in range(5)]
                etest_S1[i, j, k] = [np.linalg.norm(df_['ccm_testS1'][idx] - varss_S1.autocorrelation(5)[idx]) for idx in range(5)]

            etrain_joint[i, j, k] = [np.linalg.norm(df_['ccm_train_joint'][idx] - varss_joint.autocorrelation(5)[idx]) for idx in range(5)]
            etest_joint[i, j, k] = [np.linalg.norm(df_['ccm_test_joint'][idx] - varss_joint.autocorrelation(5)[idx]) for idx in range(5)]

            etrain_jointM1[i, j, k] = [np.linalg.norm(df_['ccm_trainM1'][idx] - varss_joint.autocorrelation(5, proj=M1_proj)[idx]) for idx in range(5)]
            etest_jointM1[i, j, k] = [np.linalg.norm(df_['ccm_testM1'][idx] - varss_joint.autocorrelation(5, proj=M1_proj)[idx]) for idx in range(5)]
        

            if df_['ccm_trainS1'].shape[-1] == 0: 
                etrain_jointS1[i, j, k] = 5*[np.nan]
                etest_jointS1[i, j, k] = 5*[np.nan]
            else:
                etrain_jointS1[i, j, k] = [np.linalg.norm(df_['ccm_testS1'][idx] - varss_joint.autocorrelation(5, proj=S1_proj)[idx]) for idx in range(5)]
                etest_jointS1[i, j, k] = [np.linalg.norm(df_['ccm_testS1'][idx] - varss_joint.autocorrelation(5, proj=S1_proj)[idx]) for idx in range(5)]

        

In [37]:
np.mean(etest_M1[0], axis=0)

array([[10.18709493,  5.93213143,  4.62710963,  4.21830198,  3.94745552],
       [11.2293733 ,  5.79794963,  5.05324065,  3.55024075,  3.3089208 ],
       [13.74877306,  7.56068938,  6.15302178,  5.51719435,  4.05322118],
       [18.36608323, 11.20093005,  9.43139297,  8.02620239,  7.26154032],
       [24.10159544, 15.69028094, 13.25797473, 11.29182826,  9.79938752]])

In [40]:
np.mean(etest_M1[0], axis=0).shape

(5, 5)

In [41]:
np.sum(np.mean(etest_M1[0], axis=0), axis=1)

array([28.91209349, 28.93972513, 37.03289974, 54.28614896, 74.14106688])

In [42]:
np.sum(np.mean(etest_jointM1[0], axis=0), axis=1)

array([28.63531864, 28.60276422, 37.42693224, 56.22008037, 78.1036549 ])

In [ ]:
# Given the loss of performance for longer lags, could look at regularization as an answer. Have Li experiment with subspace identification. What about self regression?

In [ ]:
# Rough draft of Figure 2: 


In [ ]:
# Next: Calculate the TE between M1 and S1 for different lags.

### Fitting (peanut)

In [43]:
from loaders import load_peanut

In [44]:
data_file = '/mnt/Secondary/data/peanut/data_dict_peanut_day14.obj'

In [52]:
results_list = []


for i, epoch in tqdm(enumerate(np.arange(2, 18, 2))):
    dat_HPC = load_peanut(data_file, epoch, spike_threshold=200, region='HPc')
    dat_OFC = load_peanut(data_file, epoch, spike_threshold=100, region='OFC')

    yM1 = np.squeeze(dat_HPC['spike_rates'])
    yS1 = np.squeeze(dat_OFC['spike_rates'])
    yjoint = np.hstack([yM1, yS1])

    fold_idx = 0 

    # Cross-validated marginal models
    for train_idxs, test_idxs in KFold(5).split(yM1):

        ccm_trainM1 = estimate_autocorrelation(yM1[train_idxs], 5)
        ccm_testM1 = estimate_autocorrelation(yM1[test_idxs], 5)
        ccm_trainS1 = estimate_autocorrelation(yS1[train_idxs], 5)
        ccm_testS1 = estimate_autocorrelation(yS1[test_idxs], 5)

        ccm_train_joint = estimate_autocorrelation(yjoint[train_idxs], 5)
        ccm_test_joint = estimate_autocorrelation(yjoint[test_idxs], 5)    dat_M1 = load_peanut(data_file, epoch, spike_threshold=200)



        for var_order in [1, 2, 3, 4, 5]:
            
            results = {}
            results['data_file'] = data_file
            results['fold_idx'] = fold_idx
            results['var_order'] = var_order
            results['ccm_trainM1'] = ccm_trainM1
            results['ccm_testM1'] = ccm_testM1
            results['ccm_trainS1'] = ccm_trainS1 
            results['ccm_testS1'] = ccm_testS1
            results['ccm_train_joint'] = ccm_train_joint
            results['ccm_test_joint'] = ccm_test_joint

            varmodel = VAR(estimator='ols', order=var_order)
            varmodel.fit(yM1[train_idxs])
            results['m1_coef'] = varmodel.coef_

            varmodel.fit(yS1[train_idxs])
            results['s1_coef'] = varmodel.coef_

            varmodel.fit(yjoint[train_idxs])
            results['joint_coef'] = varmodel.coef_
            results_list.append(results)

        fold_idx += 1   

0it [00:00, ?it/s]../../cov_estimation.py:142: RuntimeWarning: invalid value encountered in double_scalars
  ccm_sample_normalized[k, i, j] /= np.sqrt(ccm_sample[0, i, i] * ccm_sample[0, j, j])
Rows of VAR matrix processed: 54it [00:00, 219.35it/s]
Rows of VAR matrix processed: 194it [00:05, 35.11it/s]
Rows of VAR matrix processed: 248it [00:09, 26.20it/s]
Rows of VAR matrix processed: 54it [00:00, 78.34it/s]
Rows of VAR matrix processed: 194it [00:12, 14.96it/s]
Rows of VAR matrix processed: 248it [00:20, 11.84it/s]
Rows of VAR matrix processed: 54it [00:00, 54.50it/s]
Rows of VAR matrix processed: 194it [00:21,  8.99it/s]
Rows of VAR matrix processed: 248it [00:47,  5.27it/s]
Rows of VAR matrix processed: 54it [00:01, 41.46it/s]
Rows of VAR matrix processed: 194it [00:34,  5.54it/s]
Rows of VAR matrix processed: 248it [00:59,  4.16it/s]
Rows of VAR matrix processed: 54it [00:01, 34.19it/s]
Rows of VAR matrix processed: 194it [00:45,  4.25it/s]
Rows of VAR matrix processed: 248it [01:

KeyboardInterrupt: 

### Fitting (CV)

### Fitting (PVC)